Primero importamos las librerias necesarias


In [36]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
from itertools import chain

Importamos los datos

In [37]:
df = pd.read_csv('data/stud.csv')
df

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [38]:
df['Total_Score']=(df['math_score'] + df['reading_score'] + df['writing_score'])
df['Average_Score']=(df['Total_Score']/3)

Quiero hacer la prediccion del "Average_Score" de teniendo en cuenta el "math_score", el "reading_score" y el "writing_score"

In [39]:
x=df[['math_score',	'reading_score',	'writing_score']]
y=df[['Average_Score']]
x

,math_score,reading_score,writing_score
0,72,72,74
1,69,90,88
2,90,95,93
3,47,57,44
4,76,78,75
...,...,...,...
995,88,99,95
996,62,55,55
997,59,71,65
998,68,78,77


El autor aplica el algoritmo one hot encoding, el cual le asigna 1 y 0 a las variables categoricas para poder utilizarlas en ML

In [40]:
numeric_features = [feature for feature in df.columns if df[feature].dtype != 'O']
categorical_features = [feature for feature in df.columns if df[feature].dtype == 'O']

columnas_interes = categorical_features



valores_unicos=[]
for i in categorical_features:
    m=(df[i].unique()).tolist()
    valores_unicos.append(m)
    
    
#print(valores_unicos)


valores_unicos_planos = list(chain.from_iterable(valores_unicos))

#print(valores_unicos_planos)


for i in valores_unicos_planos:
    df[i]=0
    
    
total_filas=df.shape[0]

for i in range(total_filas):
    for c in columnas_interes:
        for m in valores_unicos_planos:
            if m == df.iloc[i][c]:
                df.at[i, m] = 1


x

,math_score,reading_score,writing_score
0,72,72,74
1,69,90,88
2,90,95,93
3,47,57,44
4,76,78,75
...,...,...,...
995,88,99,95
996,62,55,55
997,59,71,65
998,68,78,77


El autor hace one hot encoding de la siguiente forma:

In [43]:
# Create Column Transformer with 3 types of transformers
X=x
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [41]:
x

,math_score,reading_score,writing_score
0,72,72,74
1,69,90,88
2,90,95,93
3,47,57,44
4,76,78,75
...,...,...,...
995,88,99,95
996,62,55,55
997,59,71,65
998,68,78,77


In [42]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((800, 3), (200, 3))